In [ ]:
%time
# -*- coding: utf-8 -*-

from google.colab import drive
drive.mount('/content/drive')

#Importing Libraries 
import pickle
import numpy as np
import pandas as pd

with open('//content/drive/MyDrive/Ablation/data//X_train.pkl', 'rb') as f:
    datax_train = pickle.load(f)

with open('//content/drive/MyDrive/Ablation/data//Y_train.pkl', 'rb') as f:
    datay_train = pickle.load(f)
###################################################
with open('//content/drive/MyDrive/Ablation/data//X_valid.pkl', 'rb') as f:
    datax_valid = pickle.load(f)

with open('//content/drive/MyDrive/Ablation/data//Y_valid.pkl', 'rb') as f:
    datay_valid = pickle.load(f)
#####################################################
with open('//content/drive/MyDrive/Ablation/data//X_test.pkl', 'rb') as f:
    datax_test = pickle.load(f)
    
with open('//content/drive/MyDrive/Ablation/data//Y_test.pkl', 'rb') as f:
    datay_test = pickle.load(f)

#Code was used from Author Code
def prepare_data(seqs, labels, vocabsize, maxlen=None):
    """Create the matrices from the datasets.
    This pad each sequence to the same lenght: the lenght of the
    longuest sequence or maxlen.
    if maxlen is set, we will cut all sequence to this maximum
    lenght.
    This swap the axis!
    """
    # x: a list of sentences
    lengths = [len(s) for s in seqs]

    eventSeq = []

    for seq in seqs:
        t = []
        for visit in seq:
            t.extend(visit)
        eventSeq.append(t)
    eventLengths = [len(s) for s in eventSeq]


    if maxlen is not None:
        new_seqs = []
        new_lengths = []
        new_labels = []
        for l, s, la in zip(lengths, seqs, labels):
            if l < maxlen:
                new_seqs.append(s)
                new_lengths.append(l)
                new_labels.append(la)
            else:
                new_seqs.append(s[:maxlen])
                new_lengths.append(maxlen)
                new_labels.append(la[:maxlen])
        lengths = new_lengths
        seqs = new_seqs
        labels = new_labels

        if len(lengths) < 1:
            return None, None, None

    n_samples = len(seqs)
    maxlen = np.max(lengths)

    x = np.zeros((n_samples, maxlen, vocabsize)).astype('int64')
    x_mask = np.zeros((n_samples, maxlen)).astype('float64')
    y = np.zeros((n_samples, maxlen)).astype('int64')
    for idx, s in enumerate(seqs):
        x_mask[idx, :lengths[idx]] = 1
        for j, sj in enumerate(s):
            for tsj in sj:
                x[idx, j, tsj-1] = 1
    for idx, t in enumerate(labels):
        y[idx,:lengths[idx]] = t
        # if lengths[idx] < maxlen:
        #     y[idx,lengths[idx]:] = t[-1]

    return x, x_mask, y, lengths, eventLengths





CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


The code first mounts the Google Drive to access the data. It then loads the training, validation, and testing data from the pickle files.

The prepare_data() function is then defined, which prepares the input data for the CNN model. This function converts the input sequences to a matrix with each row corresponding to a sequence of the same length. Each row contains the one-hot-encoded representation of the events in the sequence. The labels are also converted to a matrix, where each row corresponds to a sequence of labels.

The code then prepares the training, testing, and validation data by calling the prepare_data() function with the required arguments.

Next, the code imports necessary libraries from scikit-learn for classification tasks such as MultiLabelBinarizer, MultiOutputClassifier, LogisticRegression, SVC, GaussianNB, LinearDiscriminantAnalysis. The code also imports the required libraries from Keras for the CNN model such as Sequential, Dense, Flatten, Dropout, Conv1D, MaxPooling1D.

After that, the code reshapes the training, validation, and testing input data to the required shape for the CNN model. Then it creates a CNN model using the Keras library, with two Conv1D layers, followed by a Dropout layer, MaxPooling1D layer, and two Dense layers with softmax activation.

Finally, it extracts the features of the training data using the model_features object and prints them.

In [ ]:
#Creating Feature set with word2Vec words that are saved with different lenghts
#Parameter 1 - Dataset that needs to vectorized
#Parameter 2 - The first number represents vocabulary size that needs to be choosen for samples
#Parameter 3 - The secong number 50 represnets max lenght that needs to used to trim extra length sentecne 
#Code was used from Author Code
training=prepare_data(datax_train,datay_train,100, 50)
testing=prepare_data(datax_test,datay_test,100,50)
validation=prepare_data(datax_valid,datay_valid,100,50)

train_X=training[0]
train_X=train_X.reshape(2800,50*100)
train_Y=datay_train
###################
valid_X=validation[0]
valid_X=valid_X.reshape(100,50*100)
valid_Y=datay_valid
####################
test_X=testing[0]
test_X=test_X.reshape(100,50*100)
test_Y=datay_test

#Importing all model libraries
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score, precision_recall_curve,roc_auc_score,accuracy_score
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



In [ ]:
YY=training[2]# extracting testing  labels for the dataset
YY=np.array(YY) # array conversion


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier

# Define the estimator and parameters for GridSearchCV
estimator = MultiOutputClassifier(LogisticRegression())
parameters = {
    'estimator__solver': ['newton-cg', 'lbfgs', 'liblinear'],
    'estimator__penalty': ['l1', 'l2', 'elasticnet'],
    'estimator__C': [0.1, 2.0, 10.0]
}

# Define the GridSearchCV object with f1-score as the scoring metric
clf = GridSearchCV(estimator, parameters, scoring='roc_auc')

# Fit the GridSearchCV object to the training data
clf.fit(train_X, YY)

# Print the best parameters and f1-score
print('Best parameters:', clf.best_params_)
print('Best roc-score:', clf.best_score_)


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Best parameters: {'estimator__C': 0.1, 'estimator__penalty': 'l2', 'estimator__solver': 'newton-cg'}
Best roc-score: 0.5431987736142688


This code calculates evaluation metrics such as accuracy, PR AUC, and ROC AUC for a given model on the testing data.

The input_testing array contains the features for the testing data and the Model is the trained machine learning model. The loop iterates over each instance of the testing data and makes predictions using the trained model.

The predict_proba method of the trained model is used to obtain the predicted probabilities of the target variable for each instance of the testing data. The probabilities are then used to calculate the PR AUC and ROC AUC scores using the average_precision_score and roc_auc_score functions from the sklearn.metrics module. If the true target variable contains at least one positive label, then the PR AUC and ROC AUC scores are calculated, otherwise, they are set to zero.

The accuracy score is calculated using the accuracy_score function from the sklearn.metrics module. The true target variable is compared with the predicted target variable obtained from the predict method of the trained model.

Finally, the calculated evaluation metrics are stored in the PR_AUC, ACC, and ROC_AUC lists, and the average scores are calculated and printed to the console.

This code performs hyperparameter tuning for a multi-output logistic regression model using GridSearchCV.

The code imports the necessary modules and defines the estimator and the parameters for GridSearchCV. The estimator is a multi-output logistic regression model, and the parameters are the hyperparameters for the model, including the regularization parameter (C), the penalty type (l1, l2, or elasticnet), and the solver used to solve the optimization problem (newton-cg, lbfgs, or liblinear).

The code then creates a GridSearchCV object with the estimator and parameters and sets the scoring metric to roc_auc. The GridSearchCV object is then fit to the training data using the fit method.

Finally, the code prints the best parameters found by the GridSearchCV object and the best roc_auc score achieved by the model.

In [ ]:
Model = MultiOutputClassifier(estimator= LogisticRegression(penalty="l2",solver="newton-cg",C=0.1)).fit(train_X, YY)
y_pred=Model.predict(test_X)

In [ ]:

YY=testing[2]
test_YY=np.array(YY)

#list to store each sample performance
PR_AUC=[]
ACC=[]
ROC_AUC=[]
#running loop over all test samples and making predictions in each case, 
#with that predictions we will find performacne scores
for i in range(test_X.shape[0]):
  inter=test_X[i].reshape(1,-1)
  y_score = Model.predict_proba(inter)
  y_pred=Model.predict(inter)
  proba=np.concatenate( y_score, axis=0)
  probabilities=proba[:, 1]
  #ROC_AUC, AU_PRC are define for only for input which has more than 1 classes 
  #in testing data for each sample , so we will calculate the score at that poin
  if test_YY[i].sum()>0:
    auc_roc=roc_auc_score(test_YY[i],probabilities)
    average_precision = average_precision_score(test_YY[i], probabilities)

  else:
    auc_roc=0 #setting zero if testing lables doesnot have more than 1 classes
    average_precision=0
  acc=accuracy_score(test_YY[i],y_pred.reshape(-1,1))
  PR_AUC.append(average_precision)
  ACC.append(acc)
  ROC_AUC.append(auc_roc)
print('ACC score is',sum(ACC)/len(ACC))
print('ROC_AUC score is',sum(ROC_AUC)/len(ROC_AUC))
print('PR_AUC score is',sum(PR_AUC)/len(PR_AUC))
%time

ACC score is 0.7980000000000002
ROC_AUC score is 0.4923752280409655
PR_AUC score is 0.282827674450073
CPU times: user 2 µs, sys: 2 µs, total: 4 µs
Wall time: 5.96 µs


In [ ]:
%whos

Variable                     Type                     Data/Info
---------------------------------------------------------------
ACC                          list                     n=100
GaussianNB                   ABCMeta                  <class 'sklearn.naive_bayes.GaussianNB'>
GridSearchCV                 ABCMeta                  <class 'sklearn.model_sel<...>on._search.GridSearchCV'>
LinearDiscriminantAnalysis   type                     <class 'sklearn.discrimin<...>earDiscriminantAnalysis'>
LogisticRegression           type                     <class 'sklearn.linear_mo<...>stic.LogisticRegression'>
Model                        MultiOutputClassifier    MultiOutputClassifier(est<...>0.1, solver='newton-cg'))
MultiLabelBinarizer          type                     <class 'sklearn.preproces<...>bel.MultiLabelBinarizer'>
MultiOutputClassifier        ABCMeta                  <class 'sklearn.multioutp<...>t.MultiOutputClassifier'>
PR_AUC                       list                     n=1